In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import librosa
# import librosa.display
import torchaudio
import matplotlib.pyplot as plt
from IPython.display import Audio

In [2]:
df_taxonomy = pd.read_csv('birdclef-2025/taxonomy.csv')
df_train_base = pd.read_csv('birdclef-2025/train.csv')

df_all = pd.merge(df_train_base, df_taxonomy[['class_name', 'primary_label']], on='primary_label', how='inner')
train_df = df_all[['primary_label','filename', 'scientific_name', 'class_name','latitude', 'longitude']]
train_df = train_df.copy()

In [3]:
aves_labels = train_df.loc[train_df['class_name']=='Aves', 'primary_label'].unique()
train_df.loc[:,'is_ave'] = train_df['primary_label'].isin(aves_labels)

In [48]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad') # type: ignore

(get_speech_timestamps, _, read_audio, *_) = utils

def highpass_filter(clean_audio, sr=32000, cutoff=800, order=6):
  import scipy.signal
  nyquist = 0.5 * sr                     
  normal_cutoff = cutoff / nyquist       
  b, a = scipy.signal.butter(order, normal_cutoff, btype='high', analog=False) # type: ignore
  y_filtered = scipy.signal.lfilter(b, a, clean_audio)
  return y_filtered

def _filter_voice(file_path, tramos = False, high_pass= False, audio_base = False):
    
  filename = file_path  
  wav, sr = torchaudio.load(filename)
  # print('Rate: ', sr)
  wav = wav.mean(dim=0)  # convertir a mono si es estéreo

  if audio_base is True:
    # print('Base')
    return wav.numpy()
    
  if tramos is True:
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sr)
    non_speech_segments = []
    prev_end = 0

    total_samples = len(wav)
    for ts in speech_timestamps:
      start = ts['start']
      end = ts['end']
        
      if prev_end < start:
        non_speech_segments.append((prev_end, start))
        prev_end = end

    # Si sobra algo al final
    if prev_end < total_samples:
      non_speech_segments.append((prev_end, total_samples))

    # recortar cada segmento:
    clean_audio = (torch.concat([wav[start:end] for start, end in non_speech_segments])).numpy()

  else:
    def fun_clean(wav, sr):
      speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sr)
      if len(speech_timestamps) != 0:

        if speech_timestamps[0]['start']<=100:
          clean_audio = (wav[speech_timestamps[0]['end']:])
        
        else:
          clean_audio = (wav[0:speech_timestamps[0]['start']])
          
      else:
        clean_audio = wav

      return clean_audio
    
    clean_audio = fun_clean(wav=wav, sr=sr)
    clean_audio = fun_clean(wav=clean_audio, sr=sr).numpy()

  if high_pass is True:
    clean_audio = highpass_filter(clean_audio=clean_audio)

  return clean_audio

def ext_chunks(audio_clean, sr, time):
  n_samples = sr * time
  chunks = []
  for i in range(0, len(audio_clean), n_samples):
    start = i
    end = i + n_samples

    if end <= len(audio_clean):
      chunk = audio_clean[start:end]
    else:   
      # padding circular
      faltan = end - len(audio_clean)
      padding = audio_clean[:faltan]
      chunk = np.concatenate([audio_clean[start:], padding])

    chunks.append(chunk)
  return np.array(chunks)

def spectogram(array_audio, n_fft=2048):
    if isinstance(array_audio, np.ndarray):
        # array_audio = array_audio.mean(dim=0) 
        dta = np.abs(librosa.stft(array_audio, n_fft=n_fft))
        D = librosa.amplitude_to_db(dta, ref=np.max)
    else:
        wav = array_audio.mean(dim=0) 
        dta = np.abs(librosa.stft(wav.numpy()))
        D = librosa.amplitude_to_db(dta, ref=np.max)
    return D


def _extract_features(audio, sr=32000):

    # MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfccs_mean = mfccs.mean(axis=1)
    mfccs_var = mfccs.var(axis=1)

    # Spectral Centroid
    sc = librosa.feature.spectral_centroid(y=audio, sr=sr)
    sc_mean = sc.mean()
    sc_var = sc.var()

    # Spectral Bandwidth 
    bw = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
    bw_mean = bw.mean()
    bw_var = bw.var()

    # Spectral Rolloff 
    srf = librosa.feature.spectral_rolloff(y=audio, sr=sr, roll_percent=0.85)
    srf_mean = srf.mean()
    srf_var = srf.var()

    # Spectral Flux 
    sf = librosa.onset.onset_strength(y=audio, sr=sr)  # proxy para flux
    sf_mean = sf.mean()
    sf_var = sf.var()

    # Spectral Flatness 
    sfm = librosa.feature.spectral_flatness(y=audio)
    sfm_mean = sfm.mean()
    sfm_var = sfm.var()

    # Zero Crossing Rate 
    zcr = librosa.feature.zero_crossing_rate(y=audio)
    zcr_mean = zcr.mean()
    zcr_var = zcr.var()

    # RMS Energy 
    rms = librosa.feature.rms(y=audio)
    rms_mean = rms.mean()
    rms_var = rms.var()

    # Vector final
    features = np.concatenate([
        mfccs_mean, mfccs_var,
        [sc_mean, sc_var],
        [bw_mean, bw_var],
        [srf_mean, srf_var],
        [sf_mean, sf_var],
        [sfm_mean, sfm_var],
        [zcr_mean, zcr_var],
        [rms_mean, rms_var]
    ])

    return features
# Nombres de las columnas
mfcc_cols = [f'mfcc_{i+1}_mean' for i in range(13)] + [f'mfcc_{i+1}_var' for i in range(13)]
extra_cols = [
    'spec_centroid_mean', 'spec_centroid_var',
    'spec_bandwidth_mean', 'spec_bandwidth_var',
    'rolloff_mean', 'rolloff_var',
    'flux_mean', 'flux_var',
    'flatness_mean', 'flatness_var',
    'zcr_mean', 'zcr_var',
    'rms_mean', 'rms_var'
]
column_names = mfcc_cols + extra_cols


Using cache found in C:\Users\overm/.cache\torch\hub\snakers4_silero-vad_master


In [5]:
# model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad') # type: ignore
# (get_speech_timestamps, _, read_audio, *_) = utils


# filename ='audio_test/audio.mp3'
# wav, sr = torchaudio.load(filename)
# print('Rate: ', sr)
# wav = wav.mean(dim=0) 

# def fun_clean(wav, sr):
#   speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sr)
#   print(speech_timestamps)
#   if len(speech_timestamps) != 0:
#     # print('True')
#     # No puede ser 0, ya que si el audio empieza unso pocos segundo en 0, no signica que no empize hablando la persona
#     if speech_timestamps[0]['start']<=100: 
#       clean_audio = (wav[speech_timestamps[0]['end']:])
    
#     else:
#       clean_audio = (wav[0:speech_timestamps[0]['start']])
      
#   else:
#     clean_audio = wav

#   return clean_audio

# clean_audio = fun_clean(wav=wav, sr=sr)
# # clean_audio = fun_clean(wav=clean_audio, sr=sr)

In [49]:
filename=train_df[train_df['class_name'] == 'Insecta']['filename']
id = f'birdclef-2025/train_audio/{filename.iloc[15]}'
clean_audio = _filter_voice(file_path=id,  tramos=False)
clean_audio = highpass_filter(clean_audio)
chunks = ext_chunks(audio_clean=clean_audio, sr=32000, time=5)
# futures = _extract_features(audio=chunks[0], sr=32000)

In [50]:
def process_audio_file(row, sr=32000, chunk_duration=5, max_chunks=3, 
                       tramos=False, apply_highpass=True, audio_base = False):

    filename = row['filename']
    is_ave = row['is_ave']
    audio_path = f'birdclef-2025/train_audio/{filename}'

    try:
        # Preprocesamiento
        clean_audio = _filter_voice(file_path=audio_path, tramos=tramos, high_pass=apply_highpass, audio_base=audio_base)
        chunks = ext_chunks(audio_clean=clean_audio, sr=sr, time=chunk_duration)

        data = []
        # used_chunks = chunks[:max_chunks] if len(chunks) > max_chunks else chunks

        # for chunk in used_chunks:
        #     feats = _extract_features(chunk, sr=sr)
        #     data.append(feats)

        for chunk in chunks:
            feats = _extract_features(chunk, sr=sr)
            data.append(feats)

        df = pd.DataFrame(data, columns=column_names)
        df['is_ave'] = is_ave

        return df

    except Exception as e:
        print(f"Error procesando {filename}: {e}")
        return pd.DataFrame()  # devuelve DataFrame vacío si hay error


In [51]:
# tomar 3 audios por especie
aves_df = train_df[train_df['is_ave'] == True]
aves_sampled = aves_df.groupby('scientific_name').apply(lambda x: x.sample(n=6, random_state=42)) \
                     .reset_index(drop=True)
no_aves_df = train_df[train_df['is_ave'] == False]
subset_df = pd.concat([aves_sampled, no_aves_df], ignore_index=True)
subset_df['is_ave'].value_counts()

False    916
True     876
Name: is_ave, dtype: int64

In [52]:
import warnings
warnings.filterwarnings('ignore')
id_labels = subset_df[['filename', 'is_ave']]
full_data = []
count = 0
for i, row in id_labels.iterrows():
    df_chunk = process_audio_file(row, tramos= False, audio_base=True)
    if not df_chunk.empty:
        full_data.append(df_chunk)
    count+=1
    print(count)

df_all = pd.concat(full_data, ignore_index=True).round(3)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [55]:
df_all.to_csv('datos_futures_B.csv', index=False)

In [56]:
hola = pd.read_csv('datos_futures_B.csv')

In [57]:
hola

,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,mfcc_9_mean,mfcc_10_mean,...,rolloff_var,flux_mean,flux_var,flatness_mean,flatness_var,zcr_mean,zcr_var,rms_mean,rms_var,is_ave
0,-457.182,-27.615,-7.902,87.386,-27.094,26.917,-2.890,10.692,2.795,-9.673,...,1.988157e+05,1.051,0.210,0.020,0.000,0.371,0.001,0.011,0.000,True
1,-451.401,-22.672,-15.683,93.317,-25.105,25.519,-4.547,15.306,-0.606,-9.735,...,9.035375e+04,0.990,0.093,0.020,0.000,0.360,0.001,0.006,0.000,True
2,-456.031,-24.423,-17.080,92.399,-24.114,19.734,-3.808,16.356,-0.961,-12.069,...,7.716837e+04,1.020,0.101,0.020,0.000,0.354,0.001,0.008,0.000,True
3,-458.103,-26.177,-10.296,92.317,-28.389,22.758,-1.497,10.952,-0.783,-6.988,...,1.290638e+05,1.010,0.099,0.023,0.000,0.360,0.001,0.008,0.000,True
4,-209.190,60.518,-10.017,24.976,-5.605,27.274,2.130,9.054,7.581,7.714,...,8.576387e+05,1.248,2.741,0.031,0.001,0.171,0.004,0.032,0.000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15176,-321.788,120.534,24.356,26.009,18.251,20.076,1.067,1.495,-1.797,3.046,...,6.143957e+06,1.121,0.370,0.006,0.000,0.059,0.003,0.027,0.000,False
15177,-316.904,123.068,25.117,25.639,20.441,19.849,0.563,-0.355,-3.059,3.136,...,8.036128e+06,1.151,0.524,0.006,0.000,0.059,0.003,0.032,0.000,False
15178,-326.459,119.831,23.998,23.606,19.797,17.965,3.780,4.140,-2.249,4.520,...,7.076049e+06,1.142,0.572,0.007,0.000,0.053,0.002,0.027,0.000,False
15179,-342.516,90.252,1.332,29.101,7.894,25.831,-1.434,-8.623,3.808,-1.608,...,8.433179e+06,1.439,4.810,0.004,0.000,0.111,0.017,0.041,0.002,False


In [60]:
hola = pd.read_csv('datos_futures_M1.csv')
hola

,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,mfcc_9_mean,mfcc_10_mean,...,rolloff_var,flux_mean,flux_var,flatness_mean,flatness_var,zcr_mean,zcr_var,rms_mean,rms_var,is_ave
0,-514.152,-104.246,-73.625,37.189,-60.273,9.274,-8.646,12.241,7.273,-5.562,...,211385.351,0.872,0.177,0.013,0.000,0.378,0.001,0.011,0.0,True
1,-521.421,-117.332,-98.145,28.401,-70.413,-1.317,-16.426,13.676,3.305,-4.069,...,96073.414,0.853,0.083,0.013,0.000,0.367,0.001,0.006,0.0,True
2,-514.620,-103.264,-84.702,40.686,-58.327,1.522,-9.620,18.308,4.356,-6.838,...,81151.517,0.844,0.096,0.013,0.000,0.359,0.001,0.008,0.0,True
3,-518.867,-108.016,-80.760,38.109,-64.648,3.070,-8.248,12.461,4.449,-1.577,...,138945.623,0.842,0.081,0.015,0.000,0.366,0.001,0.008,0.0,True
4,-305.381,-65.616,-125.595,-69.095,-80.614,-24.220,-34.174,-11.167,-6.205,1.677,...,1105116.383,1.180,2.774,0.053,0.001,0.301,0.002,0.020,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13798,-608.551,-28.464,-61.727,-41.879,-38.963,-15.905,-16.149,-5.074,-1.576,2.768,...,1578010.731,1.123,5.104,0.150,0.010,0.368,0.015,0.003,0.0,False
13799,-722.778,-127.528,-69.195,-54.447,-54.165,-1.483,-28.090,20.979,-5.028,14.033,...,745619.364,0.900,0.854,0.067,0.013,0.513,0.000,0.002,0.0,False
13800,-727.223,-102.914,-66.539,-52.135,-49.877,-5.278,-24.148,15.954,-5.332,10.780,...,852866.604,0.919,2.054,0.103,0.016,0.485,0.007,0.001,0.0,False
13801,-447.384,-20.705,-105.636,-82.166,-63.640,-40.951,-41.238,-14.648,-16.457,-1.164,...,327367.496,0.948,0.211,0.074,0.001,0.222,0.001,0.003,0.0,False


In [13]:
# wav, sr = torchaudio.load(id)
# O = spectogram(wav)
# clean_audio = _filter_voice(file_path=id, tramos=False)
# D = spectogram(clean_audio)
# high_pass = _filter_voice(file_path=id, tramos=False, high_pass = True)
# D_2 = spectogram(high_pass)

In [14]:
# plt.figure(figsize=(10, 10))

# plt.subplot(3, 1, 1)
# librosa.display.specshow(O, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
# plt.colorbar(format='%+2.0f dB')
# plt.title("Original")


# plt.subplot(3, 1, 2)
# librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
# plt.colorbar(format='%+2.0f dB')
# plt.title("Audio VAD")

# plt.subplot(3, 1, 3)
# librosa.display.specshow(D_2, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
# plt.title("Filtrado (Highpass)")
# plt.colorbar(format='%+2.0f dB')

# plt.tight_layout()
# plt.show()

# print("Audio Oiginal: ")
# display(Audio(filename=id))
# print('Audio VAD')
# display(Audio(data=clean_audio, rate=32000))
# print("Audio Pasa Altas: ")
# display(Audio(data=high_pass, rate=32000))


##### Separator (Deezer)
Esto logra separar perfectamente la voz humana y el resto lo cataloga como ruido, sin embargo, parecido a silero-vad, el audo de ruido tendría la especie y las respiraciones humanas. Sin embargo, de fondo de ruido, se alcanza a escuchar el eco de la grabación, correspondiente a la voz humana

In [15]:
# from spleeter.separator import Separator

# id = f'birdclef-2025/train_audio/{filename.iloc[1]}'

# # Inicializa el modelo de 2 fuentes
# separator = Separator('spleeter:2stems')

# # Separar el archivo
# separator.separate_to_file(id, 'output')


In [16]:
# wav, sr = torchaudio.load('output\\CSA36389\\accompaniment.wav')
# wav = wav.mean(dim=0) 
# dta = np.abs(librosa.stft(wav.numpy()))
# new = librosa.amplitude_to_db(dta, ref=np.max)

# plt.figure(figsize=(10, 5))

# librosa.display.specshow(new, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
# plt.title("Filtrado (Highpass)")
# plt.colorbar(format='%+2.0f dB')

# plt.tight_layout()
# plt.show()
# display(Audio('output\\CSA36389\\accompaniment.wav'))

In [17]:
# wav, sr = torchaudio.load('output\\CSA36389\\vocals.wav')
# wav = wav.mean(dim=0) 
# dta = np.abs(librosa.stft(wav.numpy()))
# new = librosa.amplitude_to_db(dta, ref=np.max)

# plt.figure(figsize=(10, 5))

# librosa.display.specshow(new, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
# plt.title("Filtrado (Highpass)")
# plt.colorbar(format='%+2.0f dB')

# plt.tight_layout()
# plt.show()
# display(Audio('output\\CSA36389\\vocals.wav'))